# Multi-Objective Resource Allocation Method for IoT applications in the context of Mobile Edge Computing (MEC)

## Dépendances
Import des dépendances Python nécessaires au projet.

In [1]:
import numpy as np

## Dataset
Cette partie décrit le dataset et les données utilisées pour répondre au problème décrit plus bas.

## <p style="color:red;">Partie à remplacer par le vrai dataset de Kaggle</p>

In [ ]:
# Définition des variables
n_taches = 10  # Nombre de tâches
n_noeuds = 4  # Nombre de nœuds de calcul

# Capacités de traitement maximales
C = np.array([10, 12, 8, 15])

# Puissance consommée par tâche
P = np.random.rand(n_taches, n_noeuds)

# Temps de processing par tâche
t = np.random.rand(n_taches, n_noeuds)

# Charge de travail par tâche
w = np.random.rand(n_taches)

# Paramètres de l'algorithme Firefly
alpha = 0.5  # Coefficient d'absorption
beta = 0.2  # Coefficient de randomisation
gamma = 1.0  # Facteur d'atténuation


## Définition du problème dans le code

Définition des fonctions objectives.\
On se place donc dans le cas d'un problème multi-objectifs.

In [ ]:
# Fonction objectif 1: Minimiser la consommation d'énergie totale
def f1(x):
    x_reshaped = x.reshape((n_taches, n_noeuds))
    return np.sum(P * t * x_reshaped)

# Fonction objectif 2: Minimiser le temps d'exécution total
def f2(x):
    x_reshaped = x.reshape((n_taches, n_noeuds))
    return np.sum(t * x_reshaped)

# Fonction objectif 3: Minimiser l'équilibre de charge des serveurs
# def f3(x):
#     x_reshaped = x.reshape((n_taches, n_noeuds))
#     return np.sum(t + x_reshaped)

### Définition des contraintes

In [ ]:
def constraint1(x):
    # Le nombre de vm obtenue par iot ne doit pas exceder le nombre de machines inactive sur le serveur
    return np.array([np.sum(x[i::n_noeuds]) for i in range(n_taches)]) - 1

def constraint2(x):
    # La capacité de traitement maximale de chaque nœud ne doit pas être dépassée
    return C - np.dot(w, x.reshape((n_taches, n_noeuds)))

### Fonctions Objectifs

In [ ]:
def f1_energy(position):
    return np.sum(position * demandes_taches * capacites_serveurs)

def f2_time(position):
    return np.max(np.sum(position * demandes_taches, axis=1))

In [ ]:
#Contraintes

def c1_resources(position):
    pass

def c2_nbVM(position):
    pass

In [ ]:
def fitness(position, w1, w2):
  # Consommation d'énergie
  energie = f1_energy(position)

  # Temps d'exécution
  temps = f2_time(position)

  # Fonction multiobjectives pondérée (c'est comme ça qu'on rend firefly multi-objectif)
  return w1 * energie + w2 * temps


In [ ]:
def validation(position):
  # Contrainte de ressources
  for i in range(n_serveurs):
    if np.sum(position[:, i] * demandes_taches) > capacites_serveurs[i]:
      return False

  # Contrainte de VM
  nb_vm_par_serveur = np.sum(position, axis=0)
  for i in range(n_serveurs):
    if nb_vm_par_serveur[i] > nb_vm_inactives[i]:
      return False

  return True


In [ ]:
def firefly_algorithm(
  n_taches,
  n_serveurs,
  capacites_serveurs,
  demandes_taches,
  nb_vm_inactives,
  w1=1,
  w2=1,
  w3=1,
  alpha,
  beta,
  gamma,
  iterations
  ):

  """
  Fonction implémentant l'algorithme Firefly pour l'optimisation de l'allocation de VM.

  Args:
    n_taches: Nombre de tâches.
    n_serveurs: Nombre de serveurs.
    capacites_serveurs: Capacités des serveurs.
    demandes_taches: Demandes des tâches.
    nb_vm_inactives: Nombre de VM inactives par serveur.
    w1: Poids de la consommation d'énergie.
    w2: Poids du temps d'exécution.
    w3: Poids du déséquilibre de charge.
    alpha: Coefficient d'absorption.
    beta: Coefficient de randomisation.
    gamma: Facteur d'atténuation.
    iterations: Nombre d'itérations.

  Returns:
    Meilleure position des lucioles et la meilleure valeur de la fonction d'objectif.
  """

  # Initialisation des positions des lucioles
  positions = np.random.randint(0, 2, size=(n_taches, n_serveurs))

  # Initialisation des intensités des lucioles
  intensities = np.zeros(n_taches)

  for iteration in range(iterations):

    # Mise à jour des intensités des lucioles
    intensities = np.zeros(n_taches)
    for i in range(n_taches):
      if validation(positions[i]):
        intensities[i] = fitness(positions[i])

    # Mise à jour des positions des lucioles
    for i in range(n_taches):
      if validation(positions[i]):
        for j in range(n_serveurs):
          for k in range(n_taches):
            if k != i and validation(positions[i]):
              # Calcul de la distance entre les lucioles i et k
              distance = np.linalg.norm(positions[i] - positions[k])

              # Attraction vers la luciole la plus brillante
              attraction = alpha * intensities[k] * np.exp(-gamma * distance**2)

              # Perturbation aléatoire
              perturbation = beta * np.random.rand()

              # Mise à jour de la position de la luciole
              positions[i][j] += attraction + perturbation

  # Evaluation de la meilleure solution
  best_position = positions[np.argmin(intensities)]
  best_fitness = fitness(best_position)

  return best_position, best_fitness

